In [3]:
import pandas as pd
import numpy as np
import cv2 as cv
from matplotlib import pyplot as plt
import time
from IPython.display import clear_output
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn import tree
from sklearn.decomposition import PCA
from sklearn.tree import plot_tree
from matplotlib.pylab import rcParams
from sklearn.model_selection import GridSearchCV
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
#rcParams['figure.figsize'] = 80,50

In [9]:
#Preprocess training set
data=pd.read_csv('C:/Users/X99S5/Documents/AMLS_22-23_SN18046828/Datasets/celeba/labels.csv')

labelsTrain = data["\timg_name\tgender\tsmiling"].str.split(pat="\t", n=-1, expand=True)
labelsTrain.drop(columns =[0,2], inplace = True)
labelsTrain.columns = ["img_name"  , "smiling"]
labelsTrain = labelsTrain.astype({'smiling': 'int32'})
labelsTrain = labelsTrain.astype({'img_name': 'string'})

#Preprocess testing set
data=pd.read_csv('C:/Users/X99S5/Documents/AMLS_22-23_SN18046828/Datasets/celeba_test/labels.csv')

labelsTest = data["\timg_name\tgender\tsmiling"].str.split(pat="\t", n=-1, expand=True)
labelsTest.drop(columns =[0,2], inplace = True)
labelsTest.columns = ["img_name"  , "smiling"]
labelsTest = labelsTest.astype({'smiling': 'int32'})
labelsTest = labelsTest.astype({'img_name': 'string'})


In [10]:
#Load training set################################
imagesTrain = np.zeros((5000, 150, 150,3))

for i in range(0,5000):
    image = cv.imread('C:/Users/X99S5/Documents/AMLS_22-23_SN18046828/Datasets/celeba/img/' + labelsTrain["img_name"][i])
    #image = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
    image = cv.resize(image, dsize=(150 , 150), interpolation=cv.INTER_CUBIC)
    imagesTrain[i] = image
    
# imagesTrain = imagesTrain.reshape(5000,38804)    
# imagesTrain = pd.DataFrame(imagesTrain) 


#Load Testing set################################
imagesTest = np.zeros((1000, 150, 150,3))

for i in range(0,1000):
    image = cv.imread('C:/Users/X99S5/Documents/AMLS_22-23_SN18046828/Datasets/celeba_test/img/' + labelsTest["img_name"][i])
    #image = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
    image = cv.resize(image, dsize=(150 , 150), interpolation=cv.INTER_CUBIC)
    imagesTest[i] = image
    
# imagesTest = imagesTest.reshape(1000,38804)   

# imagesTest = pd.DataFrame(imagesTest) 

In [56]:
#decision tree##### 
model = tree.DecisionTreeClassifier(max_depth=6,criterion = 'gini')
model.fit(imagesTrain,labelsTrain['smiling'])
model.score(imagesTest,labelsTest['smiling'])

0.818

In [72]:
#random forest

model = RandomForestClassifier(criterion='entropy', max_depth=5, n_estimators=140, n_jobs=-1)
model.fit(imagesTrain,labelsTrain['smiling'])
print(model.score(imagesTest,labelsTest['smiling']))

0.857


In [65]:
hyper_params = {
    'max_depth' : [2,3,4,5],
    'n_estimators' : [140, 180, 240],
    'criterion' : ['gini' , 'entropy']
}

In [68]:
model = RandomForestClassifier(n_jobs = -1)
grid=GridSearchCV(model , param_grid=hyper_params , cv=10 , n_jobs=-1 ,  verbose=10)

In [69]:
grid.fit(imagesTrain,labelsTrain['smiling'])

Fitting 10 folds for each of 24 candidates, totalling 240 fits


GridSearchCV(cv=10, estimator=RandomForestClassifier(n_jobs=-1), n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': [2, 3, 4, 5],
                         'n_estimators': [140, 180, 240]},
             verbose=10)

In [70]:
grid.best_estimator_

RandomForestClassifier(criterion='entropy', max_depth=5, n_estimators=140,
                       n_jobs=-1)

In [71]:
grid.best_score_

0.8427999999999999

In [74]:
# plot_tree(model)

In [6]:
import tensorflow as tf
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense,Conv2D,MaxPooling2D,Flatten
print("num of gpus available: ",len(tf.config.experimental.list_physical_devices('GPU')))
from keras.utils import normalize,to_categorical
from keras.preprocessing.image import ImageDataGenerator
tf.config.run_functions_eagerly(True)
#needed for tf to run

num of gpus available:  1


In [11]:
labelsTrain = labelsTrain['smiling']
labelsTest = labelsTest['smiling']

imagesTrain = normalize(imagesTrain,axis=1)
imagesTest = normalize(imagesTest,axis=1)

labelsTrain = to_categorical(labelsTrain)
labelsTest = to_categorical(labelsTest)

In [12]:
datagen_train = ImageDataGenerator()
#needed to save memory

datagen_Test = ImageDataGenerator()

In [13]:
model = Sequential()

model.add(Conv2D(32,kernel_size=(3,3),padding='valid',activation='relu',input_shape=(150,150,3)))
model.add(MaxPooling2D(pool_size=(2,2),strides=2,padding='valid'))

model.add(Conv2D(64,kernel_size=(3,3),padding='valid',activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2),strides=2,padding='valid'))

model.add(Conv2D(128,kernel_size=(3,3),padding='valid',activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2),strides=2,padding='valid'))

model.add(Flatten())

model.add( Dense(128,activation='relu') )
model.add( Dense(64,activation='relu') )
model.add( Dense(2,activation='softmax') )

In [14]:
model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['accuracy'])

In [15]:
history = model.fit( 
    datagen_train.flow(imagesTrain, labelsTrain, batch_size=100),
    epochs=15,
    validation_data=datagen_Test.flow(imagesTest, labelsTest, batch_size=100),
)

C:\Users\X99S5\AppData\Roaming\Python\Python310\site-packages\tensorflow\python\data\ops\structured_function.py:264: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


Epoch 1/15
50/50 [==============================] - 13s 124ms/step - loss: 0.0162 - accuracy: 0.9840 - val_loss: 1.4156e-07 - val_accuracy: 1.0000
Epoch 2/15
50/50 [==============================] - 6s 121ms/step - loss: 4.6373e-08 - accuracy: 1.0000 - val_loss: 1.0448e-08 - val_accuracy: 1.0000
Epoch 3/15
50/50 [==============================] - 6s 121ms/step - loss: 5.3000e-09 - accuracy: 1.0000 - val_loss: 1.9132e-09 - val_accuracy: 1.0000
Epoch 4/15
50/50 [==============================] - 6s 121ms/step - loss: 1.0836e-09 - accuracy: 1.0000 - val_loss: 4.1716e-10 - val_accuracy: 1.0000
Epoch 5/15
50/50 [==============================] - 6s 121ms/step - loss: 2.5589e-10 - accuracy: 1.0000 - val_loss: 1.1229e-10 - val_accuracy: 1.0000
Epoch 6/15
50/50 [==============================] - 6s 121ms/step - loss: 8.2238e-11 - accuracy: 1.0000 - val_loss: 4.5644e-11 - val_accuracy: 1.0000
Epoch 7/15
50/50 [==============================] - 6s 121ms/step - loss: 3.9280e-11 - accuracy: 1.0000